# IMU Data Classification

In [1]:
# Visualisation
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
import warnings

# Data processing
from sklearn.impute import KNNImputer

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn import svm

# Helper functions
from helper.helper_filter import *
from helper.helper_preprocess import *
from helper.helper_train import *

## Filter data

In [2]:
# Read the raw data from each target action and store them in a list
lqw_raw = load_data("./IMU_Data/LGW")
ramp_ascend_raw = load_data("./IMU_Data/Ramp_ascend")
ramp_descend_raw = load_data("./IMU_Data/Ramp_descend")
sit_to_stand_raw = load_data("./IMU_Data/Sit_to_stand")
stand_to_sit_raw = load_data("./IMU_Data/Stand_to_sit")

folders = [lqw_raw, ramp_ascend_raw, ramp_descend_raw, sit_to_stand_raw, stand_to_sit_raw]

In [3]:
# Drop all columns that contain sync, annotations and offset timestamps
for folder in folders:
    for file in folder:
        # Drop all columns that contain sync, annotations and offset timestamps
        file.data_filtered.drop(columns=[col for col in file.data_filtered.columns if 
                                any(info in col.lower() for info in ["sync", "offset", "annotation"])], inplace=True)
        
        # Drop all timestamp columns that are not "Shank_L_Timestamp"
        for column in file.data_filtered.columns:
            if "timestamp" in column.lower():
                if column.lower() != "shank_l_timestamp":
                    file.data_filtered.drop(columns=column, inplace=True)
        
        # Replace column name and place as the first index 
        file.data_filtered.rename(columns={'Shank_L_Timestamp': 'Timestamp'}, inplace=True)
        col = file.data_filtered.pop('Timestamp')
        file.data_filtered.insert(0, col.name, col)

In [4]:
# Replace NaN values with the k-Nearest Neighbor
for folder in folders:
    for file in folder:
        if file.data_filtered.isnull().sum().sum() > 0:
            imputer = KNNImputer(n_neighbors=5)
            file.data_filtered = pd.DataFrame(imputer.fit_transform(file.data_filtered), 
                                              columns = file.data_filtered.columns)
            
# Check if any NaN values are left
for folder in folders:
    for file in folder:
        if file.data_filtered.isnull().sum().sum() > 0: 
            print("NaN values left")

## Preprocess

In [5]:
# Remove outliers and smooth curve using a low pass filter
for folder in folders:
    for file in folder:
        # Extract sampling time
        ts = file.data_filtered["Timestamp"].diff().median() # Median sampling time

        # Remove outliers
        for name, data in file.data_filtered.items():
            if name != 'Timestamp':
                data = low_pass_filter(ts, data)

In [6]:
# Apply the slinding window technique
tw = 350        # window size
dt = 50         # window step

# Apply the moving average filter to the data and get all features
for folder in folders:
    for file in folder:
        # Apply the slinding window to the data
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
            file.data_processed = generate_features(file.data_filtered, tw, dt)

        # Drop first row where the gradient is 0
        file.data_processed = file.data_processed.iloc[1:]

In [7]:
# Combine all five actions into one dataframe and set the target labels using one-hot encoding 
iterator = 1
all_df = []

for folder in folders:    
    # Create single dataframe for action
    df = pd.DataFrame()
    df = pd.concat([file.data_processed for file in folder[:2]])

    # Add target labels
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
        df["Action"] = iterator
        iterator = iterator + 1
    
    # Add dataframe to the list
    all_df.append(df)

# Combine all dataframes into one
df = pd.concat(all_df)

In [ ]:
df.to_csv("combined_data.csv", index=False)

## Train models

In [8]:
# Split data
X = df.iloc[:, :-1]     # Input features
y = df.iloc[:, -1:]     # Target labels

# Split data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109) # 70% training and 30% test

In [9]:
# ANN
ann_model = MLPRegressor(hidden_layer_sizes=(10, 5), 
                     activation="relu" ,
                     random_state=42, max_iter=2000)

ann_model.fit(X_train, y_train)
ann_pred = ann_model.predict(X_test)

# Metrics
get_metrics(y_test, ann_pred)

C:\Users\Alex\AppData\Roaming\Python\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy:  0.8329466357308585
Precision:  0.8826694820697644
Recall:  0.8329466357308585
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.93      0.83      0.87       179
           2       0.72      0.85      0.78        85
           3       0.93      0.78      0.85        82
           4       0.86      0.98      0.91        44
           5       0.97      0.78      0.86        41
           6       0.00      0.00      0.00         0

    accuracy                           0.83       431
   macro avg       0.63      0.60      0.61       431
weighted avg       0.88      0.83      0.85       431

[[  0   0   0   0   0   0   0]
 [ 16 148  14   1   0   0   0]
 [  0  10  72   3   0   0   0]
 [  0   2  14  64   1   1   0]
 [  0   0   0   1  43   0   0]
 [  0   0   0   0   6  32   3]
 [  0   0   0   0   0   0   0]]


In [10]:
# SVM
# Create a svm Classifier
svm_model = svm.SVC(kernel='linear') # Linear Kernel
svm_model.fit(X_train, y_train)

# Predict the response for test dataset
svm_pred = svm_model.predict(X_test)

# Metrics
get_metrics(y_test, svm_pred)

Accuracy:  0.9791183294663574
Precision:  0.9794966205992132
Recall:  0.9791183294663574
              precision    recall  f1-score   support

           1       0.97      0.99      0.98       179
           2       0.96      0.96      0.96        85
           3       1.00      0.95      0.97        82
           4       1.00      1.00      1.00        44
           5       1.00      0.98      0.99        41

    accuracy                           0.98       431
   macro avg       0.99      0.98      0.98       431
weighted avg       0.98      0.98      0.98       431

[[178   1   0   0   0]
 [  3  82   0   0   0]
 [  3   1  78   0   0]
 [  0   0   0  44   0]
 [  0   1   0   0  40]]


C:\Users\Alex\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
